In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

df = pd.read_csv("../input/master.csv")
df.describe()

In [ ]:
import missingno as msno

# HDI has a much smaller count than the rest, this may mean it's highly null.
# let's visualize the nullity of our matrix
ax = msno.matrix(df)

In [ ]:
df2 = df.groupby(
    ['sex','country','age','year']
).first()[
    ['suicides_no','population']
].unstack().transpose()

df2

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
pd.options.mode.chained_assignment = None

fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(20,20))

# Base Plot: Suicides by gender across all countries
grouped = df.groupby(['country','year','sex']).agg({
        'suicides_no':['sum'],
        'population':['sum']
    }) 

grouped['suicide_rate'] = grouped[('suicides_no','sum')]/grouped[('population','sum')]*100000
grouped = grouped['suicide_rate'].reset_index()
_ = sns.lineplot(data=grouped,x='year',y='suicide_rate',hue='sex', alpha=0.5, ax=axes[0])

# First Plot: Suicides by age group over time
usa = df[df['country']=='United States']
men = usa[usa['sex']=='male']
male_suicides_by_age = men.pivot(index='year',columns='age',values='suicides/100k pop')
male_suicides_by_age.plot(title="Male Suicides per 10k by age group", ax=axes[1])

# Second Plot: Longitudinal View of Suicides by Generation

age_map = {
    '75+ years':75,
    '55-74 years': 55,
    '25-34 years': 25,
    '35-54 years': 35,
    '15-24 years': 15,
    '5-14 years' : 5
}

men.loc[:,'age_int'] = men.loc[:,'age'].apply(lambda x: age_map[x])
grouped_men = men.groupby(['generation','age_int']).agg({
    'suicides_no':['sum'],
    'population':['sum']
})

grouped_men['suicide_rate'] = grouped_men['suicides_no']/grouped_men['population']*100000
ax2 = grouped_men['suicide_rate'].reset_index().pivot(
    index='age_int',columns='generation',values='suicide_rate'
).plot(title="Suicide Rates by Generation as they Age", ax=axes[2])

fig.tight_layout()
plt.show()

## What are the countries that are anomalies?
We use DBScan to cluster the male and female suicide rates by country (we have 101 timeseries of each!)

First, we need to fill the null values..

In [ ]:
from sklearn.cluster import DBSCAN

men = grouped[grouped['sex']=='male']
men_pivoted = men.pivot(index='year',columns='country',values='suicide_rate')
men_pivoted
# clustering = DBSCAN(eps=3, min_samples=2).fit(men_pivoted)
# clustering.labels_

In [ ]:
# #####################
# Multiple Imputation #
#######################

# Linear interpolation

for country, values in men_pivoted.items():
    print(key, value)
    
# k-NN 

# Regression

In [ ]:
df[df['country']=='Antigua and Barbuda']